Importación de las librerías.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time

C:\Users\Usuario\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Definición de la variable que representa el usuario con el que cargar los datos.

In [2]:
user_id = '968973'

Creación de las funciones necesarias para el web scraping.

In [3]:
def web(page_number, user_id):
    return "https://www.filmaffinity.com/es/userratings.php?user_id=" + user_id + "&p=" + str(page_number) + "&orderby=0"

In [4]:
def number_of_pages(user_id):
    '''
    It finds the last page on the user ratings
    '''
    url = web(1, user_id)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    body = soup.body
    return int(body.find("div", {"class": "pager"}).find_all("a")[-2].string)

In [5]:
def extract_movie_info(url):
    '''
    Function to extract information from a movie given an url
        Parameters: url of the movie website
        Returns a list of information in this order: 
            [title, year, duration, country, directors, 
            guionists, musicians, photography, cast, producers, 
            genres, rating, number of raters]
    '''
    data = []
    
    # Go to movie url
    movie_page = requests.get(url)
    movie_soup = BeautifulSoup(movie_page.content)
    movie_body = movie_soup.body
    
    # Title
    print(movie_body.find(id="main-title").span.string, end = ", ")
    data.append(movie_body.find(id="main-title").span.string)
    
    # Year
    data.append(movie_body.find_all("dd", {"itemprop":"datePublished"})[0].string)
    
    # Duration (some TV series has no duration)
    try:
        data.append(movie_body.find_all("dd", {"itemprop":"duration"})[0].string)
    except:
        data.append("")
        
    # Country
    data.append(movie_body.find(id="country-img").img["alt"])
    
    # Directors
    directors = movie_body.find_all("dd", {"class":"directors"})[0].find_all("span", {"itemprop":"name"})
    directores = [director.string for director in directors]
    data.append(directores)
    
    # Since these three are by number, if we have less than 4 credits is because someone is missing, so we return all empty
    if len(movie_body.find_all("div", {"class":"credits"})) >= 4:
        # Guionists
        guionists = movie_body.find_all("div", {"class":"credits"})[1]
        guionistas = [guionista.find("span").string for guionista in guionists if guionista.find("span") != None and guionista.find("span") != -1]
        data.append(guionistas)
        # Musicians
        musicians = movie_body.find_all("div", {"class":"credits"})[2]
        musicos = [musician.find("span").string for musician in musicians if musician.find("span") != None and musician.find("span") != -1]
        data.append(musicos)
        # Photography
        photographs = movie_body.find_all("div", {"class":"credits"})[3]
        fotografos = [photograph.find("span").string for photograph in photographs if photograph.find("span") != None and photograph.find("span") != -1]
        data.append(fotografos)
    else:
        data.append([])
        data.append([])
        data.append([])
        
    # Cast (some movies has no cast so it returns an empty list)
    try:
        actors = movie_body.find_all("dd", {"class":"card-cast"})[0].find_all("span", {"itemprop":"name"})
        actores = [actor.string for actor in actors]
        data.append(actores)
    except:
        data.append([])
    
    # Production (some movies has no producers)
    try:
        producers = movie_body.find_all("dd", {"class":"card-producer"})[0].find_all("span", {"class":"nb"})
        productores = [productor.span.string for productor in producers]
        data.append(productores)
    except:
        data.append([])
    
    # Genre
    genres = movie_body.find_all("dd", {"class":"card-genres"})[0].find_all("a")
    generos = [genre.string for genre in genres]
    data.append(generos)
    
    # Rating (some movies has no rating so they return nan)
    try:
        data.append(movie_body.find(id="movie-rat-avg").string)
        data.append(movie_body.find_all("span", {"itemprop":"ratingCount"})[0].string)
    except:
        data.append(np.nan)
        data.append(np.nan)
    time.sleep(2)    
    return data

In [6]:
def extract_df_per_page(url):
    '''
    It extracts the data from each user rating page
    '''
    
    # Get the page body
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    body = soup.body
    movies_url = [poster.a["href"] for poster in body.find_all("div", {"class": "mc-poster"})]
    movies_evaluation = [rating.string for rating in body.find_all("div", {"class": "ur-mr-rat"})]

    # Create a dataframe to save the info of each movie
    df = pd.DataFrame({"titulo":[], "año":[], "duracion":[], 
                   "pais":[], "directores":[], "guionistas":[], 
                   "musicos":[], "fotografos":[], "actores":[], 
                   "productores":[], "generos":[], "nota":[], "votos": []})
    
    # Extract info from each movie url, if there is an error it shows the url
    # If there is an error saves the index to drop that evaluation
    not_added = []
    for i, url in enumerate(movies_url):
        try:
            df.loc[i] = extract_movie_info(url)
        except:
            print("{} not added".format(url))
            not_added.append(i)
            
    # Drop the evaluation of movies not added
    movies_evaluation = [ele for idx, ele in enumerate(movies_evaluation) if idx not in not_added]
        
    # Add user rating
    try:
        df["votacion"] = movies_evaluation
    except:
        print("votation of this page not added")
        df["votacion"] = ""
    
    
    return df

Creación y concatenado de los distintos datos para el dataframe final.

In [7]:
dfs = []
for page in range(number_of_pages(user_id)+1):
    print("page {} end".format(page-1))
    url = web(page, user_id)
    dfs.append(extract_df_per_page(url))
    time.sleep(5)
    if page%10 == 0:
        time.sleep(60)

page -1 end
page 0 end
Joker , 

C:\Users\Usuario\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


La ciudad de las estrellas (La La Land) , Halt and Catch Fire (Serie de TV), BoJack Horseman (Serie de TV), It's Such a Beautiful Day (Película) , Toy Story 3 , Breaking Bad (Serie de TV), Mystic River , Sleepy Hollow , Titanic , Seven (Se7en) , El rey león , Friends (Serie de TV), La lista de Schindler , La chaqueta metálica , Blade Runner , El hombre elefante , 2001: Una odisea del espacio , Matar a un ruiseñor , Psicosis , page 1 end
Senderos de gloria , Doce hombres sin piedad , La noche del cazador , La parada de los monstruos , Luces de la ciudad , The Last of Us (Serie de TV), Stranger Things 4 (Serie de TV), 30 monedas (Serie de TV), ¿Qué co#o está pasando? , Stranger Things 2 (Serie de TV), El infinito , Happiness (C), Déjame salir , Coco , Stranger Things (Serie de TV), World of Tomorrow (C), Ex Machina , Interstellar , It Follows , La gran belleza , page 2 end
Stoker , Rick y Morty (Serie de TV), Her , El cuento de la princesa Kaguya , Coherence , It's Such a Beautiful Day (

X , Avatar: El sentido del agua , El gabinete de curiosidades: La apariencia (TV), Los crímenes de la academia , El cuarto pasajero , La calle del terror. Parte 3: 1666 , Madres paralelas , Cómo meterse en un jardín (Miniserie de TV), Friends: The Reunion (TV), Britney vs. Spears , Black Phone , Halloween Kills , No mires arriba , La mujer en la ventana , Veneno (Miniserie de TV), El dilema de las redes sociales , Lo que el pulpo me enseñó , Válidas (Serie de TV), Adult Material (Miniserie de TV), The Sinner 3 (Miniserie de TV), page 37 end
Casa ajena , Un lugar tranquilo 2 , Toy Story 4 , El irlandés , Madre oscura , Elcano y Magallanes, la primera vuelta al mundo , Stranger Things 3 (Serie de TV), Historias de miedo para contar en la oscuridad , Zombieland: Mata y remata , Nosotros , Vivarium , A los gatos, ni tocarlos: Un asesino en internet (Miniserie de TV), El gran hackeo , Servant (Serie de TV), El faro , The Vast of Night , Little Joe , Ocean's 8 , Viaje al cuarto de una madre 

Días de fútbol , El despertar , Spider-Man , Mi gran boda griega , Quiero ser como Beckham , Muere otro día , 8 millas , El caso Bourne , En el filo de las olas , El coche nuevo de Mike (C), La venganza del conde de Montecristo , La dama y el vagabundo 2 , Princesa por sorpresa , El regreso de la momia , El planeta de los simios , El espinazo del diablo , Código Linux , Vanilla Sky , La fuerza del amor , La ruta hacia El Dorado , page 72 end
Pajaritos (C), American Psycho , El color de la amistad (TV), El emperador y sus locuras , X-Men , Dinosaurio , Digimon Adventure (Serie de TV), American Pie , Una historia verdadera , Magnolia , La leyenda mágica de los Leprechauns (Miniserie de TV), Toy Story 2 , Futurama (Serie de TV), Torrente, el brazo tonto de la ley , Prácticamente magia , Tú a Londres y yo a California , Rob Zombie: Dragula (Vídeo musical), Vampiros de John Carpenter , Shakespeare enamorado , El joven Hércules (Serie de TV), page 73 end
Oliver Twist (TV), Sé lo que hicistei

In [8]:
df = pd.concat(dfs)

In [9]:
df["nota"] = [re.sub(",", ".", str(nota)) for nota in df.nota]
df["nota"] = [float(re.sub("\n", "", nota)) for nota in df.nota]

In [10]:
df["votos"] = [float(re.sub("[.]", "", str(voto))) for voto in df.votos]

In [11]:
df

,titulo,año,duracion,pais,directores,guionistas,musicos,fotografos,actores,productores,generos,nota,votos,votacion
0,Joker,2019,121 min.,Estados Unidos,[Todd Phillips],"[Todd Phillips, Scott Silver]",[Hildur Guðnadóttir],[Lawrence Sher],"[Joaquin Phoenix, Robert De Niro, Zazie Beetz,...","[DC Comics, DC Entertainment, Warner Bros., Vi...","[Thriller, Drama, Crimen, DC Comics, Cómic, Pa...",8.0,69408.0,10
1,La ciudad de las estrellas (La La Land),2016,127 min.,Estados Unidos,[Damien Chazelle],[Damien Chazelle],[Justin Hurwitz],[Linus Sandgren],"[Emma Stone, Ryan Gosling, John Legend, Rosema...","[Summit Entertainment, Gilbert Films, Impostor...","[Musical, Romance, Comedia, Drama, Drama román...",7.5,58319.0,10
2,Halt and Catch Fire (Serie de TV),2014,60 min.,Estados Unidos,"[Christopher Cantwell, Christopher C. Rogers, ...","[Christopher Cantwell, Christopher C. Rogers, ...",[Paul Haslinger],"[Nelson Cragg, Evans Brown, Jeff Jur]","[Lee Pace, Scoot McNairy, Mackenzie Davis, Ker...",[AMC Studios],"[Serie de TV, Drama, Internet / Informática, T...",7.6,4517.0,10
3,BoJack Horseman (Serie de TV),2014,25 min.,Estados Unidos,"[Raphael Bob-Waksberg, JC Gonzalez, Amy Winfre...","[Raphael Bob-Waksberg, Joe Lawson, Peter Knigh...","[Grouplove, Jesse Novak]","[ShadowMachine Films, Tornante Company, Netfli...",[],"[ShadowMachine Films, Tornante Company, Netfli...","[Serie de TV, Animación, Comedia, Drama, Comed...",8.0,11520.0,10
4,It's Such a Beautiful Day (Película),2012,62 min.,Estados Unidos,[Don Hertzfeldt],[Don Hertzfeldt],[Varios],[Don Hertzfeldt],[],[Bitter Films],"[Animación, Drama, Comedia dramática, Animació...",7.7,2781.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,Ali G anda suelto,2002,88 min.,Reino Unido,[Mark Mylod],"[Sacha Baron Cohen, Dan Mazer]",[Adam Fenton],[Ashley Rowe],"[Sacha Baron Cohen, Michael Gambon, Charles Da...","[Universal Pictures, Studiocanal, Working Titl...","[Comedia, Comedia absurda, Película de culto, ...",5.2,36100.0,1
17,Darkened Room (C),2002,8 min.,Estados Unidos,[David Lynch],[David Lynch],[Angelo Badalamenti],[David Lynch],"[Jordan Ladd, Etsuko Shikata, Cerina Vincent]",[Davidlynch.com],"[Intriga, Drama, Surrealismo, Cine experimenta...",5.0,641.0,1
18,¡Ja me maaten...!,2000,86 min.,España,[Juan Muñoz],"[Luis Lázaro, Juan Muñoz, Cristóbal Ruiz]","[Eugenio Fernández, Juan Muñoz]",[Fernando Arribas],"[Juan Muñoz, José Carabias, Paz Aragón, Ángel ...","[Don Oriol S.A, World Entertaiments]",[Comedia],2.0,4379.0,1
19,Battle Royale,2000,114 min.,Japón,[Kinji Fukasaku],"[Kenta Fukasaku, Koushun Takami]",[Masamichi Amano],[Katsumi Yanagishima],"[Aki Maeda, Tatsuya Fujiwara, Chiaki Kuriyama,...","[Kobi Co., AM Associates, Fukasaku-gumi, Gaga ...","[Thriller, Acción, Colegios & Universidad, Pel...",6.5,35461.0,1


Exportación del dataframe en formato csv.

In [84]:
df.to_csv("./puntuacionesFA.csv", index = False)